In [0]:
%python
import requests
import datetime
import json
import os

class Collector:

    def __init__(self, url):
        self.url = url  # Define a URL base da API
        self.instance = url.strip("/").split("/")[-1]  # Extrai o último segmento da URL para nomear a instância

    def get_endpoint(self, **kwargs):
        """Realiza uma requisição GET para a API com os parâmetros fornecidos"""
        resp = requests.get(self.url, params=kwargs)
        return resp  # Retorna a resposta da requisição
    
    def save_data(self, data):
        """Salva os dados obtidos da API em um arquivo JSON no DBFS"""
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Cria um timestamp para nomear os arquivos
        data['ingestion_date'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Adiciona a data de ingestão nos dados
        
        directory = "dbfs/mnt/pokemon/pokemon/"  # Define o diretório de salvamento
        filename = f"{directory}/{now}.json"  # Define o caminho completo do arquivo
        
        # Cria o diretório caso não exista
        os.makedirs(directory, exist_ok=True)
        
        # Salva os dados no arquivo JSON
        with open(filename, 'w') as open_file:
            json.dump(data, open_file)

    def get_and_save(self, **kwargs):
        """Obtém os dados da API e os salva localmente"""
        resp = self.get_endpoint(**kwargs)  # Faz a requisição para a API
        if resp.status_code == 200:  # Verifica se a requisição foi bem-sucedida
            data = resp.json()  # Converte a resposta para JSON
            self.save_data(data)  # Salva os dados no arquivo JSON
            return data  # Retorna os dados obtidos
        else:
            return {}  # Retorna um dicionário vazio em caso de erro
    
    def auto_exec(self, limit=100):
        """Executa a coleta automática de dados de forma paginada"""
        offset = 0  # Define o deslocamento inicial para a paginação

        while True:
            data = self.get_and_save(limit=limit, offset=offset)  # Obtém e salva os dados da API

            if data["next"] is None:  # Verifica se há uma próxima página
                break  # Se não houver, encerra a coleta
                
            offset += limit  # Atualiza o deslocamento para a próxima página

# Define a URL da API para coletar os dados dos Pokémon
url = "https://pokeapi.co/api/v2/pokemon/"

# Cria uma instância do coletor para a API do Pokémon
collector = Collector(url)

# Executa a coleta de dados uma única vez e salva os resultados
data = collector.get_and_save()  # Obtém os dados e os salva

collector.save_data(data)  # Salva os dados obtidos

In [0]:
dbutils.fs.mkdirs("dbfs/mnt/pokemon/pokemon/")

In [0]:
dbutils.fs.ls("dbfs/mnt/pokemon/pokemon")